In [1]:
from scipy.ndimage import affine_transform
import tifffile as tiff
import os
import numpy as np
import copy
from pathlib import Path
import sys
import matplotlib.pyplot as plt
import cv2
PIPELINE_ROOT =  Path().resolve().parent
sys.path.append(PIPELINE_ROOT.as_posix())

sys.path.append('/data/pipeline/src/')

# from image_manipulation.pipeline_process import Pipeline
# from image_manipulation.elastix_manager import create_downsampled_transforms
# from utilities.utilities_mask import equalized, normalize_image


In [3]:
"""This module takes care of the CZI file management. We used to use the bftool kit 
(https://www.openmicroscopy.org/)
which is a set of Java tools, but we opted to use a pure python library that
can handle CZI files. https://github.com/AllenCellModeling/aicspylibczi
"""
import os
from PIL import Image
from aicspylibczi import CziFile
from aicsimageio import AICSImage


# from library.image_manipulation.file_logger import FileLogger
# from library.utilities.utilities_process import write_image
from library.utilities.utilities_mask import equalized


class CZIManager():
    """Methods to extract meta-data from czi files using AICSImage module (Allen Institute)
    """
    
    def __init__(self, czi_file):
        """Set up the class with the name of the file and the path to it's location.

        :param czi_file: string of the name of the CZI file
        """
        
        self.czi_file = czi_file
        self.file = CziFile(czi_file)

        # LOGFILE_PATH = os.environ["LOGFILE_PATH"]
        # super().__init__(LOGFILE_PATH)


    def extract_metadata_from_czi_file(self, czi_file, czi_file_path):
        """This will parse the xml metadata and return the relevant data.

        :param czi_file: string of the CZI file name
        :param czi_file_path: string of the CZI path
        :return: dictionary of the metadata
        """

        czi_aics = AICSImage(czi_file_path)
        total_scenes = czi_aics.scenes

        czi_meta_dict = {}
        scenes = {}
        for idx, scene in enumerate(total_scenes):
            czi_aics.set_scene(scene)
            dimensions = (czi_aics.dims.X, czi_aics.dims.Y)
            channels = czi_aics.dims.C

            print(f"CZI file: {czi_file}, scene: {czi_aics.current_scene}, dimensions: {dimensions}, channels: {channels}")

            scenes[idx] = {
                "scene_name": czi_aics.current_scene,
                "channels": channels,
                "dimensions": dimensions,
            }

        czi_meta_dict[czi_file] = scenes
        return czi_meta_dict
    
    def get_scene_dimension(self, scene_index):
        """Gets the bounding box size of the scene

        :param scene_index: integer of the scene index
        :return: x,y,width and height of the bounding box
        """

        scene = self.file.get_scene_bounding_box(scene_index)
        return scene.x, scene.y, scene.w, scene.h
    
    def get_scene(self, scene_index, channel, scale=1):
        """Gets the correct scene from the slide

        :param scene_index: integer of the scene index
        :param channel: integer of the channel
        :param scale: integer of the scale. Usually either 1 or 16 (full or downsampled)
        :return: the scene  
        """

        region = self.get_scene_dimension(scene_index)
        return self.file.read_mosaic(region=region, scale_factor=scale, C=channel - 1)[0]

from tifffile import imread, imwrite
def write_image(file_path:str, data, message: str = "Error") -> None:
    """Writes an image to the filesystem
    """
    
    try:
        imwrite(file_path, data)
        #cv2.imwrite(file_path, data)
    except Exception as e:
        print(message, e)
        print("Unexpected error:", sys.exc_info()[0])
        try:
            imwrite(file_path, data)
        except Exception as e:
            print(message, e)
            print("Unexpected error:", sys.exc_info()[0])
            sys.exit()

def extract_tiff_from_czi(file_key):
    """Gets the TIFF file out of the CZI and writes it to the filesystem

    :param file_key: a tuple of: czi_file, output_path, scenei, channel, scale
    """
    czi_file, output_path, scenei, channel, scale = file_key
    czi = CZIManager(czi_file)
    data = None
    try:
        data = czi.get_scene(scale=scale, scene_index=scenei, channel=channel)
    except Exception as e:
        message = f" ERROR READING SCENE {scenei} CHANNEL {channel} [extract_tiff_from_czi] IN FILE {czi_file} to file {os.path.basename(output_path)} {e}"
        print(message)
        czi.logevent(message)
        return

    message = f"ERROR WRITING SCENE - [extract_tiff_from_czi] FROM FILE {czi_file} -> {output_path}; SCENE: {scenei}; CHANNEL: {channel} ... SKIPPING"
    write_image(output_path, data, message=message)



def extract_png_from_czi(file_key, normalize = True):
    """This method creates a PNG file from the TIFF file. This is used for viewing
    on a web page.
    
    :param file_key: tuple of _, infile, outfile, scene_index, scale
    :param normalize: a boolean that determines if we should normalize the TIFF
    """

    _, infile, outfile, scene_index, scale = file_key

    czi = CZIManager(infile)
    try:
        data = czi.get_scene(scene_index=scene_index, channel=1, scale=scale)
        if normalize:
            data = equalized(data)
        im = Image.fromarray(data)
        im.save(outfile)
    except Exception as e:
        message = f"ERROR READING SCENE - [extract_png_from_czi] IN FILE {infile} ERR: {e}"
        print(message)
        czi.logevent(message)


In [5]:
animal = 'DKBC008'
channel  = 1
downsample = True
scaling_factor = 32
debug = True
task = 'extract'
czi_file = 'DKBC008_cord_slide004_loc1_2024_07_23_axion2.czi'

DIR = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DKBC008/czi_rescans'

czi_file_path = Path(DIR, czi_file)
print(f'ANALYZING FILE: {czi_file}')

czi = CZIManager(czi_file_path)
czi.extract_metadata_from_czi_file(czi_file, czi_file_path)

ANALYZING FILE: DKBC008_cord_slide004_loc1_2024_07_23_axion2.czi
CZI file: DKBC008_cord_slide004_loc1_2024_07_23_axion2.czi, scene: ScanRegion0, dimensions: (14917, 9377), channels: 2


{'DKBC008_cord_slide004_loc1_2024_07_23_axion2.czi': {0: {'scene_name': 'ScanRegion0',
   'channels': 2,
   'dimensions': (14917, 9377)}}}

In [11]:
import os
output = '/scratch/tiff_out/'
scenei = 0
channel = 1
scale = 1 #1 is full resolution, 16 is downsampled
czi_file = Path(DIR, czi_file)

os.makedirs(output, exist_ok=True)
output_path = Path(output, f'{czi_file.stem}_scene{scenei}_channel{channel}_scale{scale}.tiff')
print(f'DATA WILL BE SAVED TO: {output_path}')

file_key = (czi_file_path, output_path, scenei, channel, scale)
extract_tiff_from_czi(file_key)

DATA WILL BE SAVED TO: /scratch/tiff_out/DKBC008_cord_slide004_loc1_2024_07_23_axion2_scene0_channel1_scale1.tiff


In [11]:
import xml.etree.ElementTree as ET
from aicspylibczi import CziFile
from aicsimageio import AICSImage

animal = 'DK78'
channel  = 1
downsample = True
scaling_factor = 32
debug = True
task = 'extract'
#czi_file = 'DK78_slide059_2022_05_25_axion2.czi'

czi_file = Path('/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK41a/czi/DK41_slide062_2020_09_26__9332.czi')

czi = AICSImage(czi_file)
metadata_element = czi.metadata
metadata_str = ET.tostring(metadata_element, encoding='unicode')

#print(metadata_str)

# Save metadata_str to a file
metadata_file_path = 'metadata_output.txt'  # Define the file path where you want to save the metadata


with open(metadata_file_path, 'w', encoding='utf-8') as file:
    file.write(metadata_str)

print(f"Metadata saved to {metadata_file_path}")

Metadata saved to metadata_output.txt


In [37]:
import json

json_meta = {}
tracing_data = []
metadata_str = ET.tostring(metadata_element, encoding='unicode')
parsed_metadata = ET.fromstring(metadata_str)

channels_activated = parsed_metadata.findall(".//Channel[@IsActivated='true']")

for channel_idx, channel in enumerate(channels_activated):
    channel_name = channel.get('Name')
    channel_description = channel.get('Description')
    print('C'+str(channel_idx+1), channel_name,channel_description)
    
    tracing_data.append({"id": str(channel_idx+1), "mode": "dye", "description": channel_description, "channel_name": "C"+str(channel_idx+1)})
    
    # channel_str = ET.tostring(channel, encoding='unicode')
    # print(channel_str)

    #print(f'{"id": "1", "mode": "dye", "description": "NeurotraceBlue", "channel_name": "C"+str(channel_idx+1)}')
    #json_meta["Neuroanatomical_tracing"] = {str(channel_idx)}

print(tracing_data)




# Define the base structure for the data
# data = {
#     "Neuroanatomical_tracing": {}
# }

# # Define the tracing data to be added
# tracing_data = [
#     {"id": "1", "mode": "dye", "description": "NeurotraceBlue", "channel_name": "C1"},
#     {"id": "3", "mode": "virus", "description": "GFP", "channel_name": "C3"}
# ]

# # Loop through each item in tracing_data and add it to the data dictionary
# for item in tracing_data:
#     data["Neuroanatomical_tracing"][item["id"]] = {
#         "mode": item["mode"],
#         "description": item["description"],
#         "channel_name": item["channel_name"]
#     }

# # Specify the file name
# file_name = 'meta-data.json'

# # Writing the data to the file
# with open(file_name, 'w') as file:
#     json.dump(data, file, indent=4)

C1 CFP 
C2 AF568 
C3 TuYFP 
[{'id': '1', 'mode': 'dye', 'description': '', 'channel_name': 'C1'}, {'id': '2', 'mode': 'dye', 'description': '', 'channel_name': 'C2'}, {'id': '3', 'mode': 'dye', 'description': '', 'channel_name': 'C3'}]


In [41]:
tracing_data

[{'id': '1', 'mode': 'dye', 'description': '', 'channel_name': 'C1'},
 {'id': '2', 'mode': 'dye', 'description': '', 'channel_name': 'C2'},
 {'id': '3', 'mode': 'dye', 'description': '', 'channel_name': 'C3'}]

In [29]:
metadata_str = ET.tostring(metadata_element, encoding='unicode')
parsed_metadata = ET.fromstring(metadata_str)

xy_resolutions = parsed_metadata.findall(".//Items")

for xml_element in xy_resolutions:
    
    # Find the Distance elements for X and Y
    x_distance_element = xml_element.find(".//Distance[@Id='X']")
    y_distance_element = xml_element.find(".//Distance[@Id='Y']")
    
    if x_distance_element is not None:
        x_value = x_distance_element.find('Value').text
        x_unit = x_distance_element.find('DefaultUnitFormat').text
        print(f"x_res: {x_value} {x_unit}")
    else:
        print("x_res: Not found")
    
    if y_distance_element is not None:
        y_value = y_distance_element.find('Value').text
        y_unit = y_distance_element.find('DefaultUnitFormat').text
        print(f"y_res: {y_value} {y_unit}")
    else:
        print("x_res: Not found")

x_res: 3.25E-07 µm
y_res: 3.25E-07 µm


In [ ]:
<Items>
    <Distance Id="X">
     <Value>3.25E-07</Value>
     <DefaultUnitFormat>µm</DefaultUnitFormat>
    </Distance>
    <Distance Id="Y">
     <Value>3.25E-07</Value>
     <DefaultUnitFormat>µm</DefaultUnitFormat>
    </Distance>
   </Items>
